In [ ]:
import numpy as np
import skimage as sk
import skimage.io as skio

In [ ]:
# verify installation
# print(np.__version__)
# print(sk.__version__)

In [ ]:
# all raw input files
images = ["cathedral.jpg",
          "church.tif",
          "emir.tif",
          "harvesters.tif",
          "icon.tif",
          "lady.tif",
          "melons.tif",
          "monastery.jpg",
          "onion_church.tif",
          "sculpture.tif",
          "self_portrait.tif",
          "three_generations.tif",
          "tobolsk.jpg",
          "train.tif"]

In [ ]:
small_images = ["cathedral.jpg", "monastery.jpg", "tobolsk.jpg"]

In [ ]:
large_images = ["vases.tiff",
                "bridge.tiff",
                "flowers.tiff",
                "church.tif",
                "harvesters.tif",
                "icon.tif",
                "lady.tif",
                "melons.tif",
                "onion_church.tif",
                "sculpture.tif",
                "self_portrait.tif",
                "three_generations.tif",
                "train.tif"]

In [ ]:
# file util functions
def get_file(fname): 
    # read in the image
    impath = f"../../data/{fname}"
    imname = fname.split(".")[0]
    im = skio.imread(impath)

    # convert to double
    im = sk.img_as_float(im)

    return imname, im

def save_file(imname, im_out):
    fname = f"../images/{imname}_combined.jpg"
    print(f"Saving output as {fname}...")
    skio.imsave(fname, im_out)

In [ ]:
# image functions
def separate_color_channels(im):
    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]
    assert(np.shape(b) == np.shape(g) == np.shape(r))

    return b, g, r

def create_color_image(r, g, b):
    im_out = np.dstack([r, g, b])
    im_out = sk.img_as_ubyte(im_out)
    return im_out

def display_image(im):
    skio.imshow(im)
    skio.show()

# calculates the sum of squared differences (ssd) between images a and b
def ssd(a, b):
    diff = np.diff([a, b], axis=0)
    squared_diff = np.power(diff, 2)
    ssd = np.sum(squared_diff)
    
    return ssd

# shift an image by some dx, dy
def shift(im, dx, dy):
    shifted_im = np.roll(im, dy, axis=0)
    shifted_im = np.roll(shifted_im, dx, axis=1)
    
    return shifted_im

def crop(r, g, b, margin=10):
    return r[margin:-margin, margin:-margin], g[margin:-margin, margin:-margin], b[margin:-margin, margin:-margin] 

In [ ]:
# single scale algorithm
# finds the displacement vector that best aligns image b to image a
def find_displacement(a, b, window=15):    
    # the only 90% of the pixels in the image are used to calculate the ssd, centered around the middle of the image
    midpoint_x, midpoint_y = len(a[0]) // 2, len(a) // 2
    dx, dy = np.floor(len(a[0]) * .4).astype(int), np.floor(len(a) * .4).astype(int)
    left, right, top, bottom = midpoint_x - dx, midpoint_x + dx, midpoint_y - dy, midpoint_y + dy
    
    # iterate and search over the range [-window, window]
    min_ssd = np.inf
    best_i, best_j = 0, 0
    for i in range(-window, window+1):
        for j in range(-window, window+1):
            b_shifted = np.roll(b, i, axis=0)
            b_shifted = np.roll(b_shifted, j, axis=1)
            current_ssd = ssd(a[top:bottom, left:right], b_shifted[top:bottom, left:right])
            
            if current_ssd < min_ssd:
                min_ssd = current_ssd
                best_i, best_j = i, j

    print(f"Found ({best_j}, {best_i}) to be the best displacement vector with ssd={min_ssd}")
    return best_j, best_i

In [ ]:
# image pyramid algorithm
# recurive function that creates an image pyramid to do a coarse to fine displacement search
def find_displacement_recursive(a, b, window=3):
    prev_i, prev_j = 0, 0
    rescale_threshold = 100
    
    if len(a[0]) > rescale_threshold or len(a) > rescale_threshold:
        
        a_downsized = sk.transform.rescale(a, 1/2, anti_aliasing=True)
        b_downsized = sk.transform.rescale(b, 1/2, anti_aliasing=True)
        
        prev_j, prev_i = find_displacement_recursive(a_downsized, b_downsized)
        
        # multiply to account for rescaling and apply best results from recursive calls
        prev_i *= 2
        prev_j *= 2
        b = shift(b, prev_j, prev_i)

    # the only 90% of the pixels in the image are used to calculate the ssd, centered around the middle of the image
    midpoint_x, midpoint_y = len(a[0]) // 2, len(a) // 2
    dx = np.floor(len(a[0]) * .45).astype(int)
    dy = np.floor(len(a) * .45).astype(int)
    left, right, top, bottom = midpoint_x - dx, midpoint_x + dx, midpoint_y - dy, midpoint_y + dy

    a_cropped = a[top:bottom, left:right]
    b_cropped = b[top:bottom, left:right]
    
    # iterate and search over the range [-window, window]
    min_ssd = np.inf
    best_i, best_j = 0, 0
    for i in range(-window, window):
        for j in range(-window, window):
            b_shifted = np.roll(b_cropped, i, axis=0)
            b_shifted = np.roll(b_shifted, j, axis=1)
            current_ssd = ssd(a_cropped, b_shifted)
            
            if current_ssd < min_ssd:
                min_ssd = current_ssd
                best_i, best_j = i, j

    best_i += prev_i
    best_j += prev_j
    print(f"Found ({best_j}, {best_i}) to be the best displacement vector with ssd={min_ssd}")
    return best_j, best_i

In [ ]:
# bells and whistles
# auto-contrast
def contrast(r, g, b):
    mask = np.zeros(np.shape(r), dtype=np.int8)
    top, bottom, left, right = np.floor(len(r) * 0.1).astype(int), np.floor(len(r) * 0.9).astype(int), np.floor(len(r[0]) * 0.1).astype(int), np.floor(len(r[0]) * 0.9).astype(int)
    mask[top:bottom, left:right] = 1
    r_contrasted = sk.exposure.equalize_hist(r)
    g_contrasted = sk.exposure.equalize_hist(g)
    b_contrasted = sk.exposure.equalize_hist(b)
    return r_contrasted, g_contrasted, b_contrasted

In [ ]:
# integration functions
def run_small():
    for input in small_images:
        imname, im = get_file(input)
        b, g, r = separate_color_channels(im)
    
        dx, dy = find_displacement(b, r)
        ar = shift(r, dx, dy)
        dx, dy = find_displacement(b, g)
        ag = shift(g, dx, dy)

        im_out = create_color_image(ar, ag, b)
        save_file(imname, im_out)
        display_image(im_out)

def run_large(): 
    for input in large_images:
        imname, im = get_file(input)
        b, g, r = separate_color_channels(im)
        cropped_r, cropped_g, cropped_b = crop(r, g, b, margin=250)
    
        dx, dy = find_displacement_recursive(cropped_b, cropped_r)
        ar = shift(r, dx, dy)
        dx, dy = find_displacement_recursive(cropped_b, cropped_g)
        ag = shift(g, dx, dy)

        im_out = create_color_image(ar, ag, b)
        save_file(imname, im_out)
        display_image(im_out)

def run_contrast(): 
    for input in ["church.tif", "harvesters.tif", "icon.tif", "lady.tif"]:
        imname, im = get_file(input)
        b, g, r = separate_color_channels(im)
        cropped_r, cropped_g, cropped_b = crop(r, g, b, margin=250)
    
        dx, dy = find_displacement_recursive(cropped_b, cropped_r)
        ar = shift(r, dx, dy)
        dx, dy = find_displacement_recursive(cropped_b, cropped_g)
        ag = shift(g, dx, dy)

        # channel contrast
        imname = imname + "_contrasted"
        ar, ag, b = contrast(ar, ag, b)
        im_out = create_color_image(ar, ag, b)
        display_image(im_out)
        save_file(imname, im_out)

# emir.tif turns out better if the b and g channels are aligned to r, instead of aligning to b like the other images
def run_emir_bad():
    imname, im = get_file("emir.tif")
    imname = "emir_bad"
    b, g, r = separate_color_channels(im)
    cropped_r, cropped_g, cropped_b = crop(r, g, b, margin=250)

    # align to b
    dx, dy = find_displacement_recursive(cropped_b, cropped_r)
    ar = shift(r, dx, dy)
    dx, dy = find_displacement_recursive(cropped_b, cropped_g)
    ag = shift(g, dx, dy)

    im_out = create_color_image(ar, ag, b)
    save_file(imname, im_out)
    display_image(im_out)

def run_emir():
    imname, im = get_file("emir.tif")
    b, g, r = separate_color_channels(im)
    cropped_r, cropped_g, cropped_b = crop(r, g, b, margin=250)

    # align to r
    dx, dy = find_displacement_recursive(cropped_r, cropped_b)
    ab = shift(b, dx, dy)
    dx, dy = find_displacement_recursive(cropped_r, cropped_g)
    ag = shift(g, dx, dy)

    im_out = create_color_image(r, ag, ab)
    save_file(imname, im_out)
    display_image(im_out)

In [ ]:
run_small()
run_large()
run_emir_bad()
run_emir()
run_contrast()